In [11]:
import pandas as pd
import sys
sys.path.append('..')
from src.data_loader import DataLoader
from src.feature_engineering import FeatureEngineering

# Load data
data_loader = DataLoader()
data = data_loader.load_data('C:/Users/dell/Desktop/10Academy/gokada-delivery-optimization/data/driver_locations_during_request.csv')


In [16]:

fixed_location = (6.602207, 3.270465)  # Example of a fixed location

data = FeatureEngineering.create_temporal_features(data)
data = FeatureEngineering.calculate_distances(data, fixed_location)
features_to_normalize = ['lat', 'lng', 'hour_of_day', 'day_of_week', 'month', 'distance_to_fixed_location']
data = FeatureEngineering.normalize_features(data, features_to_normalize)
# Feature Correlation Heatmap
data = FeatureEngineering.normalize_features(data, features_to_normalize)
# FeatureEngineering.plot_heatmap(data)
data.head()

,id,order_id,driver_id,driver_action,lat,lng,created_at,updated_at,hour_of_day,day_of_week,month,distance_to_fixed_location
0,1,392001,243828,accepted,1.103259,-2.032688,NaT,NaN,NaN,NaN,NaN,0.243277
1,2,392001,243588,rejected,0.934717,-1.714322,NaT,NaN,NaN,NaN,NaN,0.111701
2,3,392001,243830,rejected,1.002008,-1.820457,NaT,NaN,NaN,NaN,NaN,0.141224
3,4,392001,243539,rejected,1.002145,-1.844040,NaT,NaN,NaN,NaN,NaN,0.163859
4,5,392001,171653,rejected,1.220380,-1.688913,NaT,NaN,NaN,NaN,NaN,-0.217183
